In [1]:
### caser tensorflow implementation with MAN data preprocessing
### bu script guzel calisiyor

import numpy as np 
import pandas as pd 
# from google.colab import drive
# drive.mount('/content/gdrive')
import os 
import scipy.sparse as sp

train_root = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml_1M/ratings.dat'


import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Lambda,Embedding, Conv1D,Conv2D
from tensorflow.keras.layers import MaxPool1D,Dropout, Lambda, Concatenate, Flatten
from tensorflow.keras.layers import Multiply,Add
import matplotlib.pyplot as plt
from HelperFunctions import time_conv,time_sorter,unique_definer, corpus_creator
from HelperFunctions import sequencer_multi,input_label_maker,train_neg_maker

In [2]:
df = pd.read_csv(train_root,delimiter = '::',header = None)
df.columns = ['user_id','item_id','ratings','timestamp']


# test_df = pd.read_csv(test_root, delimiter = ' ',header = None)
# test_df.columns = ['user_id','item_id','ratings']


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
df.head(3)

,user_id,item_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [4]:
df['timestamp'] = time_conv(df['timestamp'])
df = time_sorter(df,'user_id', 'timestamp')
df.head(3)

,user_id,timestamp,item_id,ratings
0,1,2000-12-31 22:00:19,3186,4
1,1,2000-12-31 22:00:55,1270,5
2,1,2000-12-31 22:00:55,1721,4


In [5]:
np.max(df['item_id'].values)

3952

In [6]:
df['item_id'].nunique()

3706

In [7]:
unique_users, num_unique_user = unique_definer(df,'user_id')
unique_items, num_unique_item = unique_definer(df,'item_id')

In [8]:
num_unique_item

3706

In [9]:
item_2enc, enc_2item = corpus_creator(unique_items, 1)
user_2enc, enc_2user = corpus_creator(unique_users, 1)

In [10]:
df['user_id_enc'] = df['user_id'].map(enc_2user)
df['item_id_enc'] = df['item_id'].map(enc_2item)

In [11]:
df

,user_id,timestamp,item_id,ratings,user_id_enc,item_id_enc
0,1,2000-12-31 22:00:19,3186,4,1,1
1,1,2000-12-31 22:00:55,1270,5,1,2
2,1,2000-12-31 22:00:55,1721,4,1,3
3,1,2000-12-31 22:00:55,1022,5,1,4
4,1,2000-12-31 22:01:43,2340,3,1,5
...,...,...,...,...,...,...
1000204,6040,2001-08-10 14:40:29,2917,4,6040,1249
1000205,6040,2001-08-10 14:41:04,1921,4,6040,371
1000206,6040,2001-08-10 14:41:04,1784,3,6040,90
1000207,6040,2001-08-10 14:41:26,161,3,6040,465


In [12]:
df_enc = df[['user_id_enc','item_id_enc']].copy()

In [13]:
sequenced_df_enc = sequencer_multi(df_enc,'user_id_enc')
sequenced_df_enc.head(3)

,user_id_enc,item_id_enc
0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,2,"[54, 55, 56, 57, 58, 59, 60, 10, 61, 62, 63, 6..."
2,3,"[66, 63, 176, 92, 177, 16, 178, 179, 180, 181,..."


In [14]:
#### 80 % of the item data will be used as training data 
#### 20 % of the item data will be used as test data

In [15]:
def train_test_maker(df,col):
    train_seq = []
    test_seq = []
    for i in range(len(df)):
        x = (len(df['item_id_enc'][i]))
        train_ratio = int(np.round(x*0.8))
        test_ratio  = np.round(x*0.2)
        # print(train_ratio)
        train_seq.append(df['item_id_enc'][i][:train_ratio])
        test_seq.append(df['item_id_enc'][i][train_ratio:])

    return(train_seq, test_seq)

In [16]:
train_seq, test_seq = train_test_maker(sequenced_df_enc,'item_id_enc')

In [17]:
sequenced_df_enc['train_seq_enc'] = train_seq
sequenced_df_enc['test_seq_enc'] = test_seq


In [18]:
sequenced_df_enc.head(3)

,user_id_enc,item_id_enc,train_seq_enc,test_seq_enc
0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]"
1,2,"[54, 55, 56, 57, 58, 59, 60, 10, 61, 62, 63, 6...","[54, 55, 56, 57, 58, 59, 60, 10, 61, 62, 63, 6...","[150, 151, 152, 153, 154, 155, 156, 157, 158, ..."
2,3,"[66, 63, 176, 92, 177, 16, 178, 179, 180, 181,...","[66, 63, 176, 92, 177, 16, 178, 179, 180, 181,...","[2, 203, 88, 128, 204, 48, 205, 206, 207, 208]"


In [19]:
n = 6
var2 =[]
u_id = [] ### user_index
for i in range(len(sequenced_df_enc)):
    x = sequenced_df_enc['train_seq_enc'][i]
    var1 = []
    var_u1 = []
    for j in range(len(x)):
        batch = x[j:j+n]
        if len(batch) == n :
            var1.append(batch)
            u_id.append(sequenced_df_enc['user_id_enc'][i])
    var2.append(var1)
var2 = np.concatenate(var2)
var2 = [list(i) for i in var2]

In [20]:
window_df = pd.DataFrame({'user_id_enc':u_id, 'item_id_enc':var2})

test_indices =(np.where(window_df.duplicated(subset=['user_id_enc']) == False)[0]-1)[1:]
# test_indices
train_df = window_df.drop(test_indices).reset_index(drop = True)
test_df = window_df.iloc[test_indices].reset_index(drop = True)

In [21]:
window_df.head(3)

,user_id_enc,item_id_enc
0,1,"[1, 2, 3, 4, 5, 6]"
1,1,"[2, 3, 4, 5, 6, 7]"
2,1,"[3, 4, 5, 6, 7, 8]"


In [22]:
last_item_train, mid_items_train = input_label_maker(train_df,'item_id_enc',6, True)
last_item_test, mid_items_test = input_label_maker(test_df,'item_id_enc',6, True)

train_df['input_items_enc'] = mid_items_train
train_df['label_items'] = last_item_train

test_df['input_items_enc'] = mid_items_test
test_df['label_items_enc'] = last_item_test
test_df['test_set_enc'] = sequenced_df_enc['test_seq_enc']
# test_df = test_df.drop(columns = ['label_items_enc','item_id_enc'])

In [23]:
test_df.head(3)

,user_id_enc,item_id_enc,input_items_enc,label_items_enc,test_set_enc
0,1,"[37, 38, 39, 40, 41, 42]","[37, 38, 39, 40, 41]",42,"[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]"
1,2,"[144, 145, 146, 147, 148, 149]","[144, 145, 146, 147, 148]",149,"[150, 151, 152, 153, 154, 155, 156, 157, 158, ..."
2,3,"[34, 198, 199, 200, 201, 202]","[34, 198, 199, 200, 201]",202,"[2, 203, 88, 128, 204, 48, 205, 206, 207, 208]"


In [24]:
final_test_set_enc = []
for i in range(len(test_df)):
    x = (int(test_df['label_items_enc'][i]))
    y = (test_df['test_set_enc'][i])
    y.insert(0,x)
    # final_test_set_enc.append()

In [25]:
train_df.head(3)

,user_id_enc,item_id_enc,input_items_enc,label_items
0,1,"[1, 2, 3, 4, 5, 6]","[1, 2, 3, 4, 5]",6
1,1,"[2, 3, 4, 5, 6, 7]","[2, 3, 4, 5, 6]",7
2,1,"[3, 4, 5, 6, 7, 8]","[3, 4, 5, 6, 7]",8


In [26]:
# us_neg = test_negative_maker(1, train_df, unique_items)

In [27]:
def test_negative_maker(num_neg,df):
    num_neg = num_neg
    neg3 = []
    ui= 0
    us = []
    neg2 = []
    for i in df['item_id_enc']:
        pos_set = i
  
        neg1 = []
        for j in range(num_neg):
    
            neg_candidate = np.random.randint(1,len(unique_items))

            while neg_candidate == pos_set:

                neg_candidate = np.random.randint(1,len(unique_items))
            us.append(ui)
      
            neg1.append((neg_candidate))
        neg2.append(np.array(neg1))

    return neg2

In [28]:
train_negs = test_negative_maker(1,train_df)

In [29]:
train_df['train_negs_enc']= train_negs

In [30]:
len(train_df)

763954

In [31]:
test_df.head(3)

,user_id_enc,item_id_enc,input_items_enc,label_items_enc,test_set_enc
0,1,"[37, 38, 39, 40, 41, 42]","[37, 38, 39, 40, 41]",42,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]"
1,2,"[144, 145, 146, 147, 148, 149]","[144, 145, 146, 147, 148]",149,"[149, 150, 151, 152, 153, 154, 155, 156, 157, ..."
2,3,"[34, 198, 199, 200, 201, 202]","[34, 198, 199, 200, 201]",202,"[202, 2, 203, 88, 128, 204, 48, 205, 206, 207,..."


In [32]:
train_df.head(3)

,user_id_enc,item_id_enc,input_items_enc,label_items,train_negs_enc
0,1,"[1, 2, 3, 4, 5, 6]","[1, 2, 3, 4, 5]",6,[102]
1,1,"[2, 3, 4, 5, 6, 7]","[2, 3, 4, 5, 6]",7,[2528]
2,1,"[3, 4, 5, 6, 7, 8]","[3, 4, 5, 6, 7]",8,[3458]


In [33]:
train_negs_enc = [int(i) for i in train_df['train_negs_enc']]
train_df['train_negs_enc'] = train_negs_enc
train_df.head(3)

,user_id_enc,item_id_enc,input_items_enc,label_items,train_negs_enc
0,1,"[1, 2, 3, 4, 5, 6]","[1, 2, 3, 4, 5]",6,102
1,1,"[2, 3, 4, 5, 6, 7]","[2, 3, 4, 5, 6]",7,2528
2,1,"[3, 4, 5, 6, 7, 8]","[3, 4, 5, 6, 7]",8,3458


In [34]:

TARGET_NUM = 1
batch_size = 4096
L=5
d=16
d_prime=4 
drop_ratio=0.05
num_factors = 5
# num_factors = 10
dims = num_factors

In [35]:

y_dummy = tf.ones( [len(train_df),1])

train_sequences = [np.array(i)  for i in (train_df['input_items_enc'])]
test_sequences = [np.array(i)  for i in (test_df['input_items_enc'])]

def expander1(x):
    return (x[:,:,:,tf.newaxis])
    
def seqer2(x):
    return (tf.squeeze(x, axis=2))

def seqer1(x):
    return (tf.squeeze(x, axis=1))

user_input = tf.keras.Input(shape = (1,), name = 'user_input')  ### kullanildi
seq_input = tf.keras.Input(shape = (L,), name = 'seq_input')  ### kullanildi

seq_embedding = Embedding(num_unique_item+1,num_factors,
                          name = 'sequence_embedding')

seq_embedding_layer = seq_embedding(seq_input)

seq_embedding_l = tf.expand_dims(seq_embedding_layer, 3,
                               name = 'sequence_embedding_expander')

user_embedding = Embedding(num_unique_user+1, num_factors,
                               name = 'user_embedding')

user_embedding_layer = user_embedding(user_input)                               

h = [i + 1 for i in range(L)]
fc1_dim_v = d_prime * num_factors
# out_v = Conv2D(4,(L,1), name = 'convo_v')(seq_embedding_l)
out_v = Conv2D(d_prime,(L,1), name = 'convo_v')(seq_embedding_l)
out_v = Flatten(name = 'flatten_convo_v')(out_v)
out_hs = []
for i in (h):
  seq_mod = tf.keras.Sequential([
            Conv2D(d, (i, num_factors)),
            # Conv2D(16, (i, num_factors)),
            Lambda(seqer2),
            MaxPool1D(L - i + 1),
            Lambda(seqer1)                      
          ])
  out_hs.append(seq_mod(seq_embedding_l))
out_h = tf.concat(out_hs, axis=1, name ='sequential_out') # galiba axis 2 olmali
out = tf.concat([out_v, out_h], axis = 1, name ='convo_concat')
dropout_layer = Dropout(0.5, name = 'dropout_layer')(out)
z = Dense(dims, name = 'dense_layer', activation = 'relu')(dropout_layer)
flat_users = Flatten(name = 'flatten')(user_embedding_layer)
x = tf.concat([z, flat_users], axis=1, name = 'concat_layer_x')

#### step1

doublet_input = tf.keras.Input(shape = (None,),
                               dtype=tf.int64, name = 'doublet_input')
W = Embedding(num_unique_item+1, num_factors*2,
                              name = 'W_embedding')## buyuk W
b_embedding = Embedding(num_unique_item+1, 1,
                               name = 'b_embedding')## kucuk b  

W_layer = W(doublet_input)## buyuk W
b_embedding_layer = b_embedding(doublet_input)## kucuk b  


#### step2

res = tf.matmul(W_layer,tf.expand_dims(x,axis=-1))                         
res = res+b_embedding_layer
res = tf.squeeze(res,axis =2)

@tf.function
def identity_loss(y_target, y_pred):
# def identity_loss(y_target, y_pred):  
  pos, neg = tf.split(y_pred,2,1)
  positive_loss = -1*tf.math.reduce_mean(tf.math.log(tf.sigmoid(pos)))
  negative_loss = -1*tf.math.reduce_mean(tf.math.log(1- tf.sigmoid(neg)))
  total_loss = tf.math.add(positive_loss, negative_loss)
  # return tf.math.reduce_mean(y_pred)
  return total_loss
caser_model =tf.keras.Model(inputs= [
                                      user_input,
                                      seq_input,
                                      doublet_input
                                      ],
                            outputs = res,
                            # outputs = x
                            )

caser_model.compile(loss = identity_loss,optimizer = 'Adam')

caser_hist = caser_model.fit(
                            [
                              tf.constant(train_df['user_id_enc']),
                              tf.constant(train_sequences),
                              tf.constant( np.squeeze( np.dstack( [ train_df['label_items'].astype('int'),
                                                                   train_df['train_negs_enc'].astype('int') 
                                                                   ] ) ) ) ],
                y_dummy,
                epochs = 50,  
                batch_size = 512 
                )

c_model_b_emb = caser_model.get_layer('b_embedding').output
c_model_W_emb = caser_model.get_layer('W_embedding').output
c_model_x = caser_model.get_layer('tf.concat_2').output

caser_model_predict= Model(inputs = [caser_model.inputs], outputs = [c_model_b_emb, c_model_W_emb,c_model_x])

# sonuclar = []
# for i in range(len(test_df)) :
#     b_emb, W_emb, x = caser_model_predict([
#                               tf.constant([test_df['test_users'][i]]),
#                               tf.constant([test_df['test_sequences'][i]]),
#                               tf.constant(np.atleast_2d(np.arange(num_items_test)))
#                                                                 #    train_df['train_negatives'].astype('int') 
#                                         ])
    


#     # res = (x * w2).sum(1) + b2
#     preds = tf.math.top_k(tf.math.reduce_sum((x*tf.squeeze(W(np.atleast_2d(np.arange(num_items_test))), axis = 0)),
#                    axis=1) + tf.squeeze(b_embedding(np.atleast_2d(np.arange(num_items_test))) ), k=10)[1]

#     sonuclar.append(preds)



def predictor(user_id):

    # i_id = (test_df[test_df['test_users'] == user_id]['test_sequences'])
    
    b_emb, W_emb, x = caser_model_predict([
                              tf.constant([test_df['user_id_enc'][user_id]]),
                              tf.constant([test_df['input_items_enc'][user_id]]),
                              tf.constant(np.atleast_2d(np.arange(num_unique_item)))
                                                                #    train_df['train_negatives'].astype('int') 
                                        ])
    # print(b_emb.shape)
    preds = (tf.math.reduce_sum((x*tf.squeeze(W(np.atleast_2d(np.arange(num_unique_item))), axis = 0)),
                   axis=1) + tf.squeeze(b_embedding(np.atleast_2d(np.arange(num_unique_item))) ))

    
    preds = np.array(preds)
    # return np.array(preds)
    return (preds)

Epoch 1/50
1493/1493 [==============================] - 24s 15ms/step - loss: 1.0224
Epoch 2/50
1493/1493 [==============================] - 23s 15ms/step - loss: 0.9365
Epoch 3/50
1493/1493 [==============================] - 23s 15ms/step - loss: 0.8429
Epoch 4/50
1493/1493 [==============================] - 24s 16ms/step - loss: 0.7906
Epoch 5/50
1493/1493 [==============================] - 23s 16ms/step - loss: 0.7601
Epoch 6/50
1493/1493 [==============================] - 23s 16ms/step - loss: 0.7399
Epoch 7/50
1493/1493 [==============================] - 23s 15ms/step - loss: 0.7199
Epoch 8/50
1493/1493 [==============================] - 23s 15ms/step - loss: 0.6992
Epoch 9/50
1493/1493 [==============================] - 21s 14ms/step - loss: 0.6793
Epoch 10/50
1493/1493 [==============================] - 21s 14ms/step - loss: 0.6633
Epoch 11/50
1493/1493 [==============================] - 21s 14ms/step - loss: 0.6512
Epoch 12/50
1493/1493 [==============================] - 21s 14

In [45]:
import numpy as np

def apk(actual, predicted, k=np.inf):

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=np.inf):

    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [37]:
p = []
for i in range(len(test_df)):
    p.append(list(np.argsort(-predictor(i))))

In [38]:
len(p)

6039

In [39]:
# np.argsort(-np.array([5,-1,0,1,2,3]))

In [40]:
test_df.head(1)

,user_id_enc,item_id_enc,input_items_enc,label_items_enc,test_set_enc
0,1,"[37, 38, 39, 40, 41, 42]","[37, 38, 39, 40, 41]",42,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]"


In [41]:
diluted_test = []
for i in range(len(p)):
    x = p[i]
    v1 = []
    for j in range(len(x)-1):
        if x[j] not in sequenced_df_enc['train_seq_enc'][i] :
            v1.append(x[j])
    diluted_test.append(v1)

In [42]:
mapk(test_df['test_set_enc'], np.array(p))

0.03306074180483279

In [43]:
len(diluted_test)

6039

In [46]:
mapk(test_df['test_set_enc'], (diluted_test))

0.08790761478760398